In [39]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
result=train.click.values
train.drop('click', axis = 1, inplace = True)

In [4]:
size_train = train.shape[0]
size_test = test.shape[0]
train = train.append(test)

In [22]:
train.isnull().sum(axis=0)

datetime             0
siteid         1583291
offerid              0
category             0
merchant             0
countrycode          0
browserid            0
devid          2524918
dtype: int64

Cleaning data

In [7]:
train=train.drop('ID',1)

In [16]:

def browser_id_def (raw):
    if ('mozilla' in raw) or ('firefox' in raw):
        return 'mozilla firefox'
    elif ('crome' in raw):
        return 'google crome'
    elif ('internet' in raw) or ('ie' in raw):
        return 'internet explorer'
    elif (raw == null):
        return 'other'
    else:
        return raw


In [20]:
train['browserid'] = train.browserid.apply(lambda x : x.lower() if type(x) == str else '')
train.browserid =train.browserid.apply(browser_id_def)


In [25]:
train['devid'].fillna("None", inplace=True)
train['siteid'].fillna(-999, inplace=True)

In [26]:
train.isnull().sum(axis=0)

datetime       0
siteid         0
offerid        0
category       0
merchant       0
countrycode    0
browserid      0
devid          0
dtype: int64

In [27]:
# set datatime
train['datetime'] = pd.to_datetime(train['datetime'])

In [29]:
# create datetime variable
train['tweekday'] = train['datetime'].dt.weekday
train['thour'] = train['datetime'].dt.hour
train['tminute'] = train['datetime'].dt.minute

In [36]:
train.head()

,siteid,offerid,category,merchant,countrycode,browserid,devid,tweekday,thour,tminute
0,4709696.0,887235,17714,20301556,4,5,2,5,9,42
1,5189467.0,178235,21407,9434818,1,5,0,2,17,50
2,98480.0,518539,25085,2050923,0,2,2,2,12,46
3,8896401.0,390352,40339,72089744,2,5,1,1,10,18
4,5635120.0,472937,12052,39507200,3,5,0,5,16,2


In [31]:
train= train.drop('datetime',1)

In [35]:
#ccols = ['countrycode','browserid','devid']
#for col in ccols:
 #   lbl = LabelEncoder()
  #  lbl.fit(list(train[col].values)
   # train[col] = lbl.transform(list(train[col].values))
            
            
label_encoder = preprocessing.LabelEncoder() 
train['countrycode']= label_encoder.fit_transform(train['countrycode']) 
train['browserid']= label_encoder.fit_transform(train['browserid']) 
train['devid']= label_encoder.fit_transform(train['devid']) 
           

#NORMALIZATION
NOt required as it cat boost ...
tree will handle itself so 



In [37]:
train_new = train[:size_train]
test_new = train[size_train:]

MODEL TRAINING

In [40]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(train_new, result, test_size=0.10, random_state=0)

In [43]:
import lightgbm as lgb
dtrain = lgb.Dataset(Xtrain, Ytrain)
dval = lgb.Dataset(Xtest, Ytest)


In [48]:
params = { 
    'num_leaves' : 256,
    'learning_rate':0.03,
    'metric':'auc',
    'objective':'binary',
    'early_stopping_round': 40,
    'max_depth':10,
    'bagging_fraction':0.5,
    'feature_fraction':0.6,
    'bagging_seed':2017,
    'feature_fraction_seed':2017,
    'verbose' : 1
}

In [ ]:
model = lgb.train(params, dtrain,num_boost_round=500,valid_sets=dval,verbose_eval=20)

In [ ]:
preds = model.predict(test_new)

In [ ]:
sub = pd.DataFrame({'ID':test['ID'], 'click':preds})
sub.to_csv('lgb_pyst.csv', index=False)